In [ ]:
import numpy as np
from qiskit import execute, QuantumCircuit
from qiskit.test.mock import FakeParis
from qiskit.providers.aer import QasmSimulator, noise

from qiskit.ignis.mitigation.measurement import (
    MeasMitigatorGenerator,
    MeasMitigatorFitter,
    expectation_value)

# Calibration of Mitigators

## Sim noise model

In [ ]:
sim = QasmSimulator()

# Example max qubit number
num_qubits = 4

# Create readout errors
readout_errors = []
for i in range(num_qubits):
    p_error1 = (i + 1) * 0.02
    p_error0 = 2 * p_error1
    ro_error = noise.ReadoutError([[1 - p_error0, p_error0], [p_error1, 1 - p_error1]])
    readout_errors.append(ro_error)

# Readout Error only
nm = noise.NoiseModel()
for i in range(num_qubits):
    nm.add_readout_error(readout_errors[i], [i])
seed_simulator = 100

## Complete A-matrix method

In [ ]:
circuits1, metadata1, _ = MeasMitigatorGenerator(num_qubits, method='complete').run()

job_cal1 = execute(circuits1, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal1 = job_cal1.result()
complete_mitigator = MeasMitigatorFitter(result_cal1, metadata1).fit(method='complete')

### Properties of the calibrated mitigator
We can now look at properties of the calibrated mitigator

#### Mitigation Overhead

In [ ]:
delta = 0.05
print('Mitigation Overhead:', complete_mitigator.mitigation_overhead())
print('Required Shots (delta = {}): {}'.format(delta, complete_mitigator.required_shots(delta)))

#### Assignment Fidelities

We can compute the single qubit avergage assignment fidelities from the mitigator

In [ ]:
print('Assignment Fidelities')
for i in range(num_qubits):
    print('Qubit {}, F_m = {:.2}'.format(i, complete_mitigator.assignment_fidelity([i])))

### Matrix Plots

We can also plot the full Assigment and mitigation matrices A, and inverse(A)

In [ ]:
print('\n Assignment Matrix')
ax1 = complete_mitigator.plot_assignment_matrix()

print('\nFull Mitigation Matrix')
ax2 = complete_mitigator.plot_mitigation_matrix()

## Single-qubit Tensored A-matrix method

In [ ]:
circuits2, metadata2, _ = MeasMitigatorGenerator(num_qubits, method='tensored').run()

job_cal2 = execute(circuits2, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal2 = job_cal2.result()
tensored_mitigator = MeasMitigatorFitter(result_cal2, metadata2).fit(method='tensored')

### Properties of the calibrated mitigator
We can now look at properties of the calibrated mitigator

#### Mitigation Overhead

In [ ]:
delta = 0.05
print('Mitigation Overhead:', tensored_mitigator.mitigation_overhead())
print('Required Shots (delta = {}): {}'.format(delta, tensored_mitigator.required_shots(delta)))

#### Assignment Fidelities

We can compute the single qubit avergage assignment fidelities from the mitigator

In [ ]:
print('Assignment Fidelities')
for i in range(num_qubits):
    print('Qubit {}, F_m = {:.2}'.format(i, tensored_mitigator.assignment_fidelity([i])))

### Matrix Plots

We can also plot the full Assigment and mitigation matrices A, and inverse(A)

In [ ]:
print('\n Assignment Matrix')
ax1 = tensored_mitigator.plot_assignment_matrix()

print('\nFull Mitigation Matrix')
ax2 = tensored_mitigator.plot_mitigation_matrix()

## Continuous-Time Markov-Process Method

In [ ]:
circuits3, metadata3, _ = MeasMitigatorGenerator(num_qubits, method='CTMP').run()

job_cal3 = execute(circuits3, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal3 = job_cal3.result()
ctmp_mitigator = MeasMitigatorFitter(result_cal3, metadata3).fit(method='CTMP')

### Properties of the calibrated mitigator
We can now look at properties of the calibrated mitigator

#### Mitigation Overhead

In [ ]:
delta = 0.05
print('Mitigation Overhead:', ctmp_mitigator.mitigation_overhead())
print('Required Shots (delta = {}): {}'.format(delta, ctmp_mitigator.required_shots(delta)))

#### Assignment Fidelities

We can compute the single qubit avergage assignment fidelities from the mitigator

In [ ]:
#print('Assignment Fidelities')
#for i in range(num_qubits):
#    print('Qubit {}, F_m = {:.2}'.format(i, ctmp_mitigator.assignment_fidelity([i])))

### Matrix Plots

We can also plot the full Assigment and mitigation matrices A, and inverse(A)

In [ ]:
ax1 = ctmp_mitigator.plot_assignment_matrix()
ax2 = ctmp_mitigator.plot_mitigation_matrix()

# Error Mitigation Examples

### Expval (all-qubits)

In [ ]:
# Test Circuit Measure <XXXX> stabilizer
qc = QuantumCircuit(4, 4)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)
qc.h(range(4))
qc.measure(range(4), range(4))

# Simulation
shots = 10000
result_target = execute(qc, sim, shots=shots).result()
result_noise = execute(qc, sim, shots=shots, noise_model=nm).result()
counts_target = result_target.get_counts(0)
counts_noise = result_noise.get_counts(0)

# Expvals
expvals = [
    ('Target', expectation_value(counts_target)),
    ('Unmitigated', expectation_value(counts_noise)),
    ('Mitigated (Complete)', expectation_value(counts_noise, mitigator=complete_mitigator)),
    ('Mitigated (Tensored)', expectation_value(counts_noise, mitigator=tensored_mitigator)),
    ('Mitigated (CTMP)', expectation_value(counts_noise, mitigator=ctmp_mitigator)),
]
print('\nExpectation Values')
for label, expval in expvals:
    print('- {}: {:.3f} \u00B1 {:.3f}'.format(label, expval[0], expval[1]))
    #print('- {}: {:.4f}, stddev: {:0.4f}'.format(label, expval[0], expval[1]))

### Expval (all-qubits, permuted)

NOTE: This is not yet implemented for CTMP method

In [ ]:
for meas_qubits in [[0, 1, 2, 3], [0, 1, 3, 2], [1, 3, 0, 2], [3, 0, 1, 2], [2, 0, 3, 1], [3, 2, 1, 0]]:

    # Test Circuit Measure <XXXX> stabilizer
    qc = QuantumCircuit(4, 4)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    qc.h(range(4))
    qc.measure(meas_qubits, range(4))

    # Simulation
    shots = 10000

    result_target = execute(qc, sim, shots=shots, seed_simulator=seed_simulator).result()
    result_noise = execute(qc, sim, shots=shots,seed_simulator=seed_simulator,
                           noise_model=nm).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expvals = [
        ('Target', expectation_value(counts_target)),
        ('Unmitigated', expectation_value(counts_noise)),
        ('Mitigated (Complete)', expectation_value(counts_noise, 
                                                   mitigator_qubits=meas_qubits,
                                                   mitigator=complete_mitigator)),
        ('Mitigated (Tensored)', expectation_value(counts_noise,
                                                   mitigator_qubits=meas_qubits,
                                                   mitigator=tensored_mitigator)),
        ('Mitigated (CTMP)', expectation_value(counts_noise,
                                               mitigator_qubits=None,
                                               mitigator=ctmp_mitigator)),
    ]
    print('\nExpectation Values (meas qubits {})'.format(meas_qubits))
    for label, expval in expvals:
        print('- {}: {:.3f} \u00B1 {:.3f}'.format(label, expval[0], expval[1]))
        #print('- {}: {:.4f}, stddev: {:0.4f}'.format(label, expval[0], expval[1]))

### Expval Partial Qubits

NOTE: This is not yet implemented for CTMP method

In [ ]:
for qubits in [[0, 1], [0, 2], [0, 3], [3, 1], [2, 1], [3, 2], [2, 3]]:
    
    # Test Circuit
    qc = QuantumCircuit(4, 2)
    qc.h(qubits[0])
    qc.cx(qubits[0], qubits[1])
    qc.measure(qubits, [0, 1])

    # Simulation
    shots = 8000
    seed = 1101
    result_target = execute(qc, sim, shots=shots, seed_simulator=seed_simulator).result()
    result_noise = execute(qc, sim, shots=shots,seed_simulator=seed_simulator,
                           noise_model=nm).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expvals = [
        ('Target', expectation_value(counts_target)),
        ('Unmitigated', expectation_value(counts_noise)),
        ('Mitigated (Complete)', expectation_value(counts_noise, 
                                                   mitigator_qubits=qubits,
                                                   mitigator=complete_mitigator)),
        ('Mitigated (Tensored)', expectation_value(counts_noise,
                                                   mitigator_qubits=qubits,
                                                   mitigator=tensored_mitigator)),
        ('Mitigated (CTMP)', expectation_value(counts_noise,
                                               mitigator_qubits=None,
                                               mitigator=ctmp_mitigator)),
    ]
    print('\nExpectation Values (meas qubits {})'.format(meas_qubits))
    for label, expval in expvals:
        print('- {}: {:.3f} \u00B1 {:.3f}'.format(label, expval[0], expval[1]))
        #print('- {}: {:.4f}, stddev: {:0.4f}'.format(label, expval[0], expval[1]))